In [25]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver    
import time
import pandas as pd
import re
import collections
import nltk
from nltk import word_tokenize, Text, FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
wnl = nltk.WordNetLemmatizer()
import pathlib
from newspaper import Article
from urllib.parse import urlparse
# importing itertools for accumulate()
import itertools
# importing functools for reduce()
import functools
from tqdm.notebook import tqdm_notebook
import json
import praw
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import time
from tqdm import tqdm
import gc
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from datetime import datetime
from urllib.parse import urlparse
import pandas as pd

In [26]:
'''

Input : Xpath for the dropdown list 
Output: Xpath for the choices in the dropdown

'''
def getNewsContentByArticle(Article, url):
    # get content 
    try:
        content = Article(url)
        content.download()
        content.parse()
        content = content.text
    except Exception as e:
        print(e)
        content = ""
    return content

def getNewsContentByGoogle(title):
    
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        
    channels = ["channelnewsasia", "straitstime","todayonline"]
    
    # to search
    query = "news:" + title
 
    content, domain, url = "", "", ""
    
    for j in search(query, tld="co.in", num=10, stop=10, pause=2):
        url = j
        
        domain = urlparse(j).netloc
        
        isIn = [True for c in channels if c in domain]
        
        content = getNewsContentByArticle(Article,j)
        
        if not isIn == []:
            break
    
    return {'content':content, 'domain':domain,'url':url}
        

def getPostListings(driver, xpath_items):
    try:
        items = driver.find_elements("xpath", xpath_items)
    except Excpetion as e:
        print('\n Empty post listing due to ',e)
        items = []
        pass
    return items

def getDropdownChoices(driver, xpath_dropdown, xpath_choices):
    # Wait for initialize, in seconds
    wait = WebDriverWait(driver, 10)

    dropdown = wait.until(EC.visibility_of_element_located((By.XPATH, xpath_dropdown)))
    
    time.sleep(1)
    
    dropdown.click()
    
    #2. find the choices on the list
    try:
        dropdown = driver.find_elements("xpath",xpath_choices)
    except ElementClickInterceptedException:
        print('\n-- No dropdown list found --')
        pass
    
    return dropdown

def getChildElement(node, xpath):
    
    xpath = ".//descendant-or-self::" + xpath
    
    try:
        child_node = node.find_element("xpath", xpath)
    except NoSuchElementException:
        print("\n-- Unable to find the child element")
        raise
        
    return child_node


def goNextPage(driver, xpath):
    try:
        button = driver.find_element("xpath",xpath)
        button.click()
        time.sleep(1)
    except:
        #print('\n-- No element {} found --'.format(xpath))
        raise

# Links Grab Test 

In [84]:
url = "https://b.cari.com.my/forum.php?mod=forumdisplay&fid=154&page=75"

linkscache = []

def selenium_init():
    #initalise crawler option(s)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    #options.add_argument('headless')
    options.add_argument('--profile-directory=Default') 
    ROOT_DIR = pathlib.Path().absolute()
    print(ROOT_DIR)
    driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)
    driver.set_window_size(1120, 1000)
    
    return driver

def getWebElementAttribute(item, xpath, name):
    try:
        text = item.find_element("xpath", xpath).get_property(name)
    except Exception as e:
        print('\n--',e)
        text = ''
    return text

def getWebElementText(item, xpath):
    try:
        text = item.find_element("xpath", xpath).text
    except Exception as e:
        print('\n--',e)
        text = ''
    return text

# 1. Get Links and URL

In [105]:
url = "https://b.cari.com.my/forum.php?mod=forumdisplay&fid=154&page=272"

driver = selenium_init()

BEGIN_FROM = datetime.strptime('2021-12-30', '%Y-%m-%d')

END_AT = datetime.strptime('2022-12-31', '%Y-%m-%d')

COUNT = 0

THRESHOLD = 500

LINKS_CACHE = []

XPATH_CLOSE_ADS = "//div[contains(@id, 'innity_adslot_')]//a[contains(@id, 'btn_close_')]"
XPATH_NEXT_MAIN = "//div[contains(@id, 'pgt')]//div[contains(@class,'pg')]/a[text()='Next']"
XPATH_LISTING_MAIN = '//table[contains(@id, "threadlisttableid")]//tbody[contains(@id, "normalthread")]'
XPATH_URL_MAIN = './/descendant-or-self::tr//th//a[contains(@class, "s xst")]'
XPATH_DATETIME_MAIN = ".//descendant-or-self::tr//th/font/span"

driver.get(url)

SEARCHING = True

while SEARCHING:
    # wait for the page to load completely
    wait = WebDriverWait(driver, 10)
    
    page_loaded = wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    if page_loaded:
        #print("\n-- DEBUG: Page loaded successfully!",end='')
        #1. close pop-up ads
        time.sleep(2)
        try:
            expanders = driver.find_elements("xpath", XPATH_CLOSE_ADS)
            for each in expanders:
                each.click()
        except NoSuchElementException:
            pass
        
        #2. get main post 
        items = getPostListings(driver, XPATH_LISTING_MAIN)
        #post_urls = [item.find_element("xpath", XPATH_URL_MAIN).get_attribute("href") for item in items]
        for post in items:
            
            post_url = getWebElementAttribute(post, XPATH_URL_MAIN, "href")
            
            post_datetime = getWebElementText(post, XPATH_DATETIME_MAIN)
            
            post_datetime = pd.to_datetime(post_datetime, format='%d-%m-%Y %I:%M %p', errors='ignore')
            
            if post_datetime < BEGIN_FROM and post_datetime > END_AT:
                COUNT = COUNT + 1
            else:
                LINKS_CACHE.append((post_datetime, post_url))
                #print('\n\t-- {} reocords have been collected on B-Cari.'.format(len(LINKS_CACHE)),end='')

        if COUNT > THRESHOLD:
            SEARCHING = False           
        
        try:
            goNextPage(driver, XPATH_NEXT_MAIN)
        except Exception as e:
            SEARCHING = False

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_18480\579970847.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)


In [106]:
import pickle
def save_pickle_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

save_pickle_object(LINKS_CACHE,'B_CARI_LINKS_2.pikle')

In [81]:
driver = selenium_init()

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_18480\579970847.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)


In [96]:
import re

XPATH_CLOSE_ADS = "//div[contains(@id, 'innity_adslot_')]//a[contains(@id, 'btn_close_')]"
XPATH_NEXT_MAIN = "//div[contains(@id, 'pgt')]//div[contains(@class,'pg')]/a[text()='Next']"
XPATH_TITLE = "//tbody/tr/td/div[contains(@id, 'postlist')]/table[not(@class)]//font"
XPATH_LISTING_POST = "//tbody/tr/td/div[contains(@id, 'postlist')]/div[not(@class)]"

test_url = 'https://b.cari.com.my/forum.php?mod=viewthread&tid=5026374&extra=page%3D475'

driver.get(test_url)

mapping = dict()

SEARCHING = True

wait = WebDriverWait(driver, 7)

while SEARCHING:
    page_loaded = wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    if page_loaded:
        #print("\n-- DEBUG: Page loaded successfully!",end='\r')
        #1. close pop-up ads
        try:
            expanders = driver.find_elements("xpath", XPATH_CLOSE_ADS)
            for each in expanders:
                each.click()
        except NoSuchElementException:
            pass

        page_items = getPostListings(driver, XPATH_LISTING_POST)

        out = []

        for item in page_items:

            #1. cmt id
            try:
                cmt_id = item.get_property('id')
            except Exception as e:
                cmt_id = ''
            #2. cmt text
            #cmt_org_content = re.sub('\s\s+',' ', getWebElementText(item, ".//descendant-or-self::div[contains(@class, 't_fsz')]/table/tbody/tr/td//text()[not(parent::div)]"))
            try:
                cmt_org_content_node = item.find_element("xpath", " .//descendant-or-self::div[contains(@class, 't_fsz')]/table/tbody/tr/td")

                cmt_to_del = cmt_org_content_node.find_elements("xpath",".//descendant-or-self::font[(ancestor::blockquote)]")

                cmt_org_content_text = cmt_org_content_node.text

                for to_del in cmt_to_del:
                    cmt_org_content_text = re.sub(to_del.text, '', cmt_org_content_text)

                cmt_org_content_text = re.sub('\s+', ' ', cmt_org_content_text)

                mapping[cmt_org_content_text] = cmt_id

            except Exception as e:
                cmt_org_content_text = ''
                #driver.execute_script("arguments[0].removeChild(arguments[1]);", cmt_org_content, to_del)

            #3. cmt reply_to
            try:
                cmt_reply_to_ = item.find_element("xpath", " .//descendant-or-self::div[contains(@class, 't_fsz')]/table/tbody/tr/td//blockquote").text

                cmt_reply_to = (' '.join(cmt_reply_to_.split('\n')[1:])).strip()
                
                cmt_reply_to = re.sub('\s+', ' ', cmt_reply_to)
                
                print(cmt_reply_to)

                cmt_reply_to = mapping.get(cmt_reply_to,'')

            except Exception as e:
                cmt_reply_to = ''

            #4. cmt reply_to raw HTML
            try:
                cmt_reply_to_raw = item.find_element("xpath", " .//descendant-or-self::div[contains(@class, 't_fsz')]/table/tbody/tr/td//blockquote").get_attribute('innerHTML')

            except Exception as e:
                cmt_reply_to_raw = ''

            #5. cmt user
            try:
                cmt_user = item.find_element("xpath", ".//descendant-or-self::div[@class='pi']/div[contains(@class,'authi')]").text
            except Exception as e:
                cmt_user = ''
            #6. cmt published datetime
            try:
                cmt_published_datetime = item.find_element("xpath", ".//descendant-or-self::div[@class='pti']/div[contains(@class,'authi')]/em").text
                cmt_published_datetime = (re.sub('Post time\s*', '', cmt_published_datetime)).strip()
                cmt_published_datetime = pd.to_datetime(cmt_published_datetime, format='%d-%m-%Y %I:%M %p', errors='ignore')
            except Exception as e:
                cmt_published_datetime = ''

            out.append({"cmt_id": cmt_id,
                        "cmt_org_content": cmt_org_content_text,
                        "cmt_published_datetime": cmt_published_datetime,
                        "cmt_replyTo": cmt_reply_to,
                        "cmt_user" : cmt_user,
                        "cmt_replyTo_HTML": cmt_reply_to_raw
                       })

            #other - lang, source, last_modified, translated
            print('--ID:', cmt_id)
            print('--USER:', cmt_user)
            print('--DATETIME:', cmt_published_datetime)
            print('--TEXT:', cmt_org_content_text)
            print('--REPLY:',cmt_reply_to)
            print('--REPLY(RAW):', cmt_reply_to_raw)
            print()

        try:
            goNextPage(driver, XPATH_NEXT_MAIN)
        except Exception as e:
            SEARCHING = False 

--ID: post_150644594
--USER: CARI-MRO
--DATETIME: 2022-03-04 11:05:00
--TEXT: Petaling Jaya – Isu makanan mahal semakin menjadi-jadi ketika ini apabila ramai peniaga mengambil kesempatan menaikkan harga setiap menu dengan mendadak sehingga dikatakan tidak masuk akal. Ini kerana, mereka memberi alasan harga bahan mentah juga semakin meningkat dan memaksa mereka untuk menaikkan harga jualan bagi mengelak kerugian. Baru-baru ini tular di media sosial apabila seorang pengguna memuat naik hantaran memaklumkan harga bagi semangkuk mi sup ikan yang dimakan di sebuah restoran di Sibu adalah RM60. Foto: Kosmo Menurut pengguna tersebut, dia berasa terkejut apabila dimaklumkan harga makanan berkenaan ketika ingin membuat pembayaran di kaunter. Kemudian, dia memeriksa semula resit yang diberikan serta menanyakan kepada peniaga soal harga makanan yang terlalu mahal. Balas peniaga itu, mi sup yang dipesan adalah menggunakan jenis ikan patin yang berharga RM190 per kilogram. Tambah peniaga itu lagi, 

In [ ]:
#1. cmt id
try:

except Exception as e:


#2. cmt content
try:

except Exception as e:


#3. cmt replyTo
try:

except Exception as e:


#4. cmt user
try:

except Exception as e:


#5. cmt published datetime
try:

except Exception as e:

#6, lang, source, last_modified, translated